In [1]:
#YOLO 사용하기위한 라이브러리
import torch
from glob import glob
import yaml
from IPython.display import Image
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import save_model, Sequential
from tensorflow.python.keras.models import load_model

#얼굴인식 관련 모델 라이븝러ㅣ
%cd ./FaceDetect4/facenet-pytorch-master/
# MTCNN 
from facenet_pytorch import MTCNN 
import mmcv, cv2, torch
from PIL import Image, ImageDraw
from torchvision import datasets, transforms
import joblib
import numpy as np
# Dlib
import imutils
from imutils import face_utils
# Euclidean distance
from scipy.spatial import distance
from math import sqrt
import pandas as pd
import dlib
from werkzeug.datastructures import FileStorage # 대용량 파일 다루기
from tempfile import TemporaryFile
# DB
import cx_Oracle

#객체 탐지 모델 접근
%cd ../../
%cd yolov5/
import detect_simple as ds
import gc
import torch


mt_model = ''

C:\Users\smhrd\RuningMachine\FaceDetect4\facenet-pytorch-master
C:\Users\smhrd\RuningMachine
C:\Users\smhrd\RuningMachine\yolov5


In [2]:
def insert_face_distance(distance_list):
    
    LOCATION = r"C:\instantclient_21_3"
    os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수
    
    user = "suhyun"
    pw = "rhdwn123"
    url = "project-db-stu.ddns.net:1524"
    data = ""
    
    connect = cx_Oracle.connect(user, pw, url) # DB연결
    cursor=connect.cursor()
    try:
        sql = "INSERT INTO T_FACE (FACE_SEQ, EUCLIDEAN_DISTANCE) VALUES (t_face_SEQ.NEXTVAL,:1)"
        for count, dist in enumerate(distance_list):
            
            if count != (len(distance_list)-1):
                data += str(dist)+" "
#                 data = str(data)
            else:
                data += str(dist)
        
        print(f'data >>>> {data} dtype : {type(data)}')
        print(len(data))
        #cursor.execute(sql,data) # 전송
        #sql = "select * from t_face"
        datas = [data]
        cursor.execute(sql,datas)
        print("DB Insert 완료!!!")
        
#         for i in cursor:
#             print(f"cursor : {i}")

    except Exception as e:
        print(f"DB insert_error {e}")
    finally:
        cursor.close()
        connect.commit()
        connect.close()

########################


def select_face():
    select = list()
    
    LOCATION = r"C:\instantclient_21_3"
    os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수
    
    user = "suhyun"
    pw = "rhdwn123"
    url = "project-db-stu.ddns.net:1524"
    
    data = ""
    
    connect = cx_Oracle.connect(user, pw, url) # DB연결
    cursor=connect.cursor()
    
    try:
        sql = "select EUCLIDEAN_DISTANCE  from t_face"
        cursor.execute(sql)
        
        for i in cursor:
            select.append(i)

    except Exception as e:
        print(f"DB insert_error {e}")
    finally:
        cursor.close()
        connect.commit()
        connect.close()
        print("select finish")
        
    return select

In [3]:
def DLIB_model(croppedImage): # 리턴 : 좌표, nparray 타입 얼굴 이미지 저장 리스트 
    coor = []
    try:
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")
    except:
        print("Dlib Predictor read error")
    #print(">>>>>>>>>>>>>",size[0],size[1])
    
    j = np.array(croppedImage)
    image = cv2.cvtColor(j, cv2.COLOR_BGR2RGB)
    imgae = cv2.resize(image,(760,450),interpolation = cv2.INTER_CUBIC)
    image = imutils.resize(image, width=760)
    color_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    rectangle = detector(color_image, 1) # 본격적인 랜드마크 검출을 위해 DLIB 타입의 사각형 좌표 (좌상단, 우하단)
    #####여기까지 실행
    for(i, rect) in enumerate(rectangle): 
        LAND = predictor(color_image, rect) 
        LAND = face_utils.shape_to_np(LAND)
        coor.append(LAND) # 사진당 얼굴 좌표 저장
    
    return coor

def euclidean_distance_calculator(vector_dim): 
    dlist = []
    for i in range(68):
        if i != 30:
            dlist.append(distance.euclidean(vector_dim[30],vector_dim[i])) # 유클리디언 거리
    return dlist

In [4]:
def faceDetect(frame,size):
    cutted_face_location = [] 
    coordinates = []
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)
    #print("1"*5)
    frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    #print("2"*5)
    boxes, _ = mtcnn.detect(frame) # boxes : 찾은 얼굴들 좌표 / _ : 얼굴일 확률 [] >> numpy array
    draw = ImageDraw.Draw(frame)
    #print("3"*5)
    if boxes is None:
        #print("4"*5)
        return np.array(frame),0
    
    print("검출된 갯수들",len(boxes))
    print(f"실제 박스들{boxes} , 확률{_}")
    
    #if len(boxes) > 1:
        #print("가장 높은 확률 인덱스번호:",_.argmax())
    
    #print("비교>>",_[_.argmax()])
    if(_[_.argmax()] >= 0.93):
        box = boxes[_.argmax()]
        #MTCNN
        #for i, box in enumerate(boxes[_.argmax()]):
        if box is not None: # 찾아낸 얼굴이 None이 아니라면 ? / 배열이 아니기에 is not Not 으로 확인해야 한다!
            draw.rectangle(box.tolist(), outline=(0, 10, 0), width=1) 
            #print("5"*5)
            x,y,w,h=box
            #print("6"*5)
            croppedImage=frame.crop((x,y,w,h))
            try:
                if croppedImage is not None: # 반드시 예외처리
                    #print("7"*5)
                    coordinates = DLIB_model(croppedImage) # DLIB 좌표 계산
                    try:   
                        if coordinates:       
                            for land in coordinates:
                                eu = euclidean_distance_calculator(land)  # 유클리디언 거리 계산 및 DB 입력 동시 처리
                                #print("eu : ",eu)
                                #insert_face_distance(eu)
                                #print("인서트 성공!!!")

                            return np.array(croppedImage) , (_*100) # 추출된 얼굴, 얼굴일 확률 [np], 유클리디언 거리
                        else:
                            return np.array(croppedImage),0
                    except: 
                        print("*******")
                else:
                    print("이미지 없음")
            except:
                print("왜 이럼?")
        else:
            return np.array(frame),0
    else:
        return np.array(frame),0

In [14]:
# 모션모델 분별함수
def motion_model_run(frame,size):
    # 보여줄 이미지 사이즈 변경
    frame = cv2.resize(frame, (size[0],size[1]))
    #frame = cv2.resize(frame, (480,240))
    
    
    # 분류모델 및 확률 추출
    images=cv2.resize(frame,(224,224))
    images = np.array([images])
    pers=int((mt_model.predict(images)[0][1])*100)
    
    # 확률에 따라 catch로 분류
    if mt_model.predict_classes(images) == 1 :
        a="CATCH"
    else :
        a=""
    
    # 결과 시각화
    # 폰트 크기
    if size[0] >700:
        b=2    
    else :
        b=1
    
    
    # 결과 시각화
    cv2.rectangle(frame,(0,0),(int(size[0]/4.5),int(size[1]/9.5)),(255,255,255),-1)
    cv2.putText(frame, a, (int(size[0]/200),int(size[1]/13)),cv2.FONT_HERSHEY_PLAIN,b,(0,0,0),b,cv2.LINE_AA)
    cv2.putText(frame, str(pers) , (int(size[0]/7.5),int(size[1]/13)),cv2.FONT_HERSHEY_PLAIN ,b,(0,0,0),b,cv2.LINE_AA)
    cv2.putText(frame, " %", (int(size[0]/6),int(size[1]/13)),cv2.FONT_HERSHEY_PLAIN ,b,(0,0,0),b,cv2.LINE_AA)
        
    return frame , a , pers

In [6]:
def send_result_data(data):
    print("send:",data)
    return redirect('http://121.147.0.176:2000/send_result_data')

In [ ]:
from flask import Flask,render_template, request, Response, redirect, url_for, abort, stream_with_context, send_file, jsonify
from werkzeug.utils import secure_filename
import cv2
%cd yolov5/
import detect_simple as ds

app = Flask(__name__)
def gen_frames(file,model_type):
    size = (640,240)
    #업로드 한 영상을 파일 로드 하는 부분
    cap = cv2.VideoCapture(os.path.join('./',file))
    while True:
 
        # # Capture frame-by-frame. Return boolean(True=frame read correctly. )
        success, frame = cap.read()  # read the camera frame
        if not success:
            break
        else:
            #모션분류,객체탐지,얼굴인식
            if 'total' in model_type:
                #frame=cv2.resize(frame,size)
                #범용성 있게 쓰겠금 변수선언
                resultImage = ''
                #모션분류 (상민씨)
                mt_frame,mt_anwser,mt_per = motion_model_run(frame,size)
                #모션분류 결과값 처리
                #mt_result = mt_anwser+","+str(mt_per)
                #Yolov5 객체탐지 (서인원,김정식)
                yl_img , yl_result = ds.detect(mt_frame)
                #원본이미지의 각 결과값 (모션분류 결과 이미지) + 객체탐지 ( 결과 이미지 )

                #cv2.rectangle(yl_img,(0,0),(90,20),(255,255,255),-1)
                #cv2.putText(yl_img, str(mt_anwser), (10,15),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,0),1,cv2.LINE_AA)
                #print("정확도:",str(mt_per)+" %")
                #cv2.putText(yl_img, str(mt_per)+" %", (55,15),cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,0,0),1,cv2.LINE_AA)
                #=============================정상=================================
                sw_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                #상우 로직 작동 태울 것.
                croppedImage, _ = faceDetect(sw_frame,size)
                #if B:
                #이미지 좌우 합치기 
                size = (800,600)

                front_result = cv2.resize(yl_img,size)
                back_result = cv2.resize(croppedImage,size) #back : 상황 캐치  , 얼굴 탐지 
                #상하 합치기
                resultImage = np.hstack((front_result,back_result))
                #else :
                #    resultImage = yl_img
                #=============================테스트용=================================
            
                #보낼 데이터 한프레임 당
#                 sendData = {
#                     'Yolov5':yl_result, #Yolov5 결과물
#                     'Motion':mt_result, #Motion 결과물
#                     'face': _ #face 확률
#                            }
#                 print("보내기전 데이터:",sendData)
                #send_result_data(sendData)
                
            #모션 분류 모델
            if 'motion' in model_type:
                size = (1600,600)
                mt_frame,mt_anwser,mt_per = motion_model_run(frame,size)
                resultImage = mt_frame
            #객체 탐지 모델
            if 'objectDetect' in model_type:
                size = (1600,600)
                yl_img , yl_result = ds.detect(frame)
                resultImage = yl_img
                resultImage = cv2.resize(resultImage,size)
            #얼굴 인식 모델
            if 'faceDetect' in model_type:
                size = (800,600)
                sw_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                croppedImage, _  = faceDetect(sw_frame,size)
                front_result = cv2.resize(frame,size) #front : 모션분류 , 객체탐지 처리완료된 이미지
                back_result = cv2.resize(croppedImage,size) #back : 상황 캐치  , 얼굴 탐지 
                #상하 합치기
                resultImage = np.hstack((front_result,back_result))

            #GPU 사용하기위해서 메모리 제거
            gc.collect()
            torch.cuda.empty_cache()

            
            #리턴받는것
            ret, buffer = cv2.imencode('.jpg', resultImage)
            #객체인식관련 코드 
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result
            
#==================================================클라 <-> 서버 연결 정의
@app.route('/total_model', methods=["GET",'POST'])
def total_render_file():
    return render_template("total_upload.html") # 3개모델합친 페이지 html 보여주기

@app.route('/motion_model', methods=["GET",'POST'])
def motion_render_file():
    return render_template("motion_upload.html") # 모션분류용 업로드 페이지

@app.route('/object_model', methods=["GET",'POST'])
def object_render_file():
    return render_template("object_upload.html") # 객체탐지용 업로드 페이지

@app.route('/face_model', methods=["GET",'POST'])
def face_render_file():
    return render_template("face_upload.html") # 얼굴인식용 업로드 페이지

@app.route('/result_data', methods=["GET",'POST'])
def data_result_file():
    #받은 데이터 확인
    print("리퀘스트 받은 부분>>!!!!!!!!!!!!")
    return render_template("result_data.html")


#================================================== 끝

#================================================= 플라스크서버 < - > 파이썬 기능 
#파일 업로드 처리
@app.route("/total_model_fileUpload", methods=['GET',"POST"])
def file_upload():
    if request.method=='POST':
        f = request.files["file"] # html <input name= "file">        
        
        #업로드 할때 파일 이름 출력
        FILE_NAME = f.filename 
        f.save(secure_filename(f.filename)) # test_park.mp4
        
        return Response(gen_frames(f.filename,'total'),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
    
@app.route("/motion_model_fileUpload", methods=['GET',"POST"])
def motion_file_upload():
    if request.method=='POST':
        f = request.files["file"] # html <input name= "file">        
        
        #업로드 할때 파일 이름 출력
        FILE_NAME = f.filename 
        f.save(secure_filename(f.filename)) # test_park.mp4
        
        return Response(gen_frames(f.filename,'motion'),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
    
@app.route("/object_model_fileUpload", methods=['GET',"POST"])
def object_file_upload():
    if request.method=='POST':
        f = request.files["file"] # html <input name= "file">        
        
        #업로드 할때 파일 이름 출력
        FILE_NAME = f.filename 
        f.save(secure_filename(f.filename)) # test_park.mp4
        
        return Response(gen_frames(f.filename,'objectDetect'),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
    
@app.route("/face_model_fileUpload", methods=['GET',"POST"])
def face_file_upload():
    if request.method=='POST':
        f = request.files["file"] # html <input name= "file">        
        
        #업로드 할때 파일 이름 출력
        FILE_NAME = f.filename 
        f.save(secure_filename(f.filename)) # test_park.mp4
        
        return Response(gen_frames(f.filename,'faceDetect'),
                    mimetype='multipart/x-mixed-replace; boundary=frame')    

    
#================================================= 플라스크서버 < - > 파이썬 기능  끝
if __name__ == '__main__':
    global mt_model
    mt_model = load_model("./test_set/model4.h5")
    #app.debug=True
    app.run(host='121.147.0.176',port=2000,threaded = True)
    #혹여 여러 요청에 대한 연산 작업을 빠르게 처리 하기 위해 Threaded 를 사용

[WinError 2] 지정된 파일을 찾을 수 없습니다: 'yolov5/'
C:\Users\smhrd\RuningMachine\yolov5
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://121.147.0.176:2000/ (Press CTRL+C to quit)
YOLOv5  v6.0-193-gdb1f83b torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)



>>> 0
사용 모델: runs/train/300epoch/weights/best.pt
device: cuda:0


Fusing layers... 
Model Summary: 213 layers, 7371523 parameters, 0 gradients, 17.0 GFLOPs
121.147.52.193 - - [08/Feb/2022 19:44:40] "POST /total_model_fileUpload HTTP/1.1" 200 -


Inferencing and Processing Done. (0.061s)
검출된 갯수들 1
실제 박스들[[     608.07       338.3      628.31      361.24]] , 확률[    0.89293]
>>> 0


...

121.147.52.193 - - [08/Feb/2022 19:45:34] "GET /total_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:34] "GET /total_model HTTP/1.1" 200 -
C:\Users\smhrd\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:454: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
YOLOv5  v6.0-193-gdb1f83b torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 


>>> 0
사용 모델: runs/train/300epoch/weights/best.pt
device: cuda:0


Model Summary: 213 layers, 7371523 parameters, 0 gradients, 17.0 GFLOPs


Inferencing and Processing Done. (0.050s)
검출된 갯수들 1
실제 박스들[[     21.967      173.85      48.382      202.97]] , 확률[    0.95231]


121.147.52.193 - - [08/Feb/2022 19:45:36] "GET /total_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:36] "GET /total_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:37] "GET /face_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:38] "GET /object_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:39] "GET /face_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:40] "GET /face_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:40] "GET /motion_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:40] "GET /motion_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:45:47] "GET /total_model HTTP/1.1" 200 -
121.147.52.193 - - [08/Feb/2022 19:46:07] "GET /total_model HTTP/1.1" 200 -


In [ ]:
%pwd

In [ ]:
%cd RuningMachine/yolov5/